## Análisis SST Región Niño 3.4

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio de trabajo

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida29/doctorado/ssev-analytics/sst_nino34
> Directorio actual:  /media/javier/Compartida29/doctorado/ssev-analytics


In [2]:
from utils.MONGO import CONEXION
from datetime import datetime
import pandas as pd

from utils.NINO_UTILS import *

In [3]:
import locale

# Cambiando sistema horario
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

# Creando Coenxión con Mongo DB

In [4]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological',
 'meteorological',
 'estimateSSTNino34',
 'polygons',
 'SSTNino34']

# Descargando la Información SST NIÑO3.4

In [5]:
# Fecha actual
time = datetime.today().toordinal()

# Realizando consulta
sst_data = db.SSTNino34.find({"time":{"$lte":time}})

# Generando pandas dataframe
data_pandas = pd.DataFrame([file for file in sst_data])
data_pandas['periodo'] = data_pandas.time.apply(lambda x: datetime.fromordinal(x))
data_pandas['mes_year'] =  data_pandas['periodo'].dt.strftime('%B-%Y')
data_pandas.index = pd.to_datetime(data_pandas.periodo)
data_pandas.head()

,_id,time,anomalias,climatologica,mes,month,nino34_mean,time_actualizacion,year,periodo,mes_year
periodo,,,,,,,,,,,
2022-10-01,63826640b386820bbf141231,738429,-1.03,26.72,octubre,10,25.69,738485,2022,2022-10-01,octubre-2022
2022-09-01,6382663fb386820bbf14121e,738399,-1.08,26.72,septiembre,9,25.64,738485,2022,2022-09-01,septiembre-2022
2022-08-01,6382663fb386820bbf141211,738368,-0.98,26.86,agosto,8,25.87,738485,2022,2022-08-01,agosto-2022
2022-07-01,6382663fb386820bbf1411ed,738337,-0.70,27.29,julio,7,26.59,738485,2022,2022-07-01,julio-2022
2022-06-01,62e6d1d870bebdf1983a27a9,738307,-0.76,27.73,junio,6,26.97,738485,2022,2022-06-01,junio-2022


In [6]:
# Registros
print(data_pandas.shape)

(874, 11)


In [7]:
# Data disponible
print(data_pandas.index.min())
print(data_pandas.index.max())

1950-01-01 00:00:00
2022-10-01 00:00:00


### Estudio de la serie temporal

In [8]:
DIR = './sst_nino34/'

In [9]:
fig_sstt_nino34 = temperatura_nino34_plot(data_pandas)
fig_sstt_nino34.write_image(DIR + 'figures/nino34/sstt_nino34.png')

fig_sstt_nino34.show()

In [10]:
# Modelo Aditivo
fig_aditivo = seasonal_decompose_plot(data=data_pandas[['nino34_mean']].dropna().copy(),
                                        model='additive',
                                        period=12)
fig_aditivo.write_image(DIR + 'figures/nino34/descom_aditivo.png')
fig_aditivo.show()

In [11]:
# Modelo Multiplicativo
fig_multiplicativo = seasonal_decompose_plot(data=data_pandas[['nino34_mean']].dropna().copy(),
                                            model='multiplicable',
                                            period=12)
                                            
fig_multiplicativo.write_image(DIR + 'figures/nino34/descom_multiplicativo.png')
fig_multiplicativo.show()

### Comportamiento climatológico

In [12]:
# Data de anomalias climaticas
pd_perioodo_anomalias = periodo_anomalias_climaticas(data_pandas)
pd_perioodo_anomalias.head()


# Data temperatura promedio
pd_temperatura = pd.merge(data_pandas[['year','month','mes','mes_year','nino34_mean','periodo','anomalia_periodo']],
                         pd_perioodo_anomalias,
                         on=['month','anomalia_periodo'],
                         how='left')
pd_temperatura['base'] = pd_temperatura\
                            .apply(lambda x: str(int(x.range_init))+'-'+str(int(x.range_end)),1 )

pd_temperatura.head()

,year,month,mes,mes_year,nino34_mean,periodo,anomalia_periodo,date_init,date_end,climatologica,range_init,range_end,base
0,2022,10,octubre,octubre-2022,25.69,2022-10-01,15,2021,2025,26.72,1991,2020,1991-2020
1,2022,9,septiembre,septiembre-2022,25.64,2022-09-01,15,2021,2025,26.72,1991,2020,1991-2020
2,2022,8,agosto,agosto-2022,25.87,2022-08-01,15,2021,2025,26.86,1991,2020,1991-2020
3,2022,7,julio,julio-2022,26.59,2022-07-01,15,2021,2025,27.29,1991,2020,1991-2020
4,2022,6,junio,junio-2022,26.97,2022-06-01,15,2021,2025,27.73,1991,2020,1991-2020


In [13]:
# Data temperatura periodo base
pd_normalidad = pd_perioodo_anomalias.copy()
pd_normalidad['base'] = pd_normalidad\
                            .apply(lambda x: str(int(x.range_init))+'-'+str(int(x.range_end)),1 )
#pd_normalidad['mes'] = pd_normalidad['month'].apply(mese_text)
pd_normalidad['mes'] =  pd.to_datetime(pd_normalidad['month'],format='%m',errors='ignore')
pd_normalidad['mes'] =  pd_normalidad['mes'].dt.strftime('%B')
pd_normalidad.head()

,date_init,date_end,anomalia_periodo,month,climatologica,range_init,range_end,base,mes
0,1946,1950,0,1,26.18,1916,1945,1916-1945,enero
1,1946,1950,0,2,26.39,1916,1945,1916-1945,febrero
2,1946,1950,0,3,26.95,1916,1945,1916-1945,marzo
3,1946,1950,0,4,27.39,1916,1945,1916-1945,abril
4,1946,1950,0,5,27.56,1916,1945,1916-1945,mayo


In [15]:
#https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_change.shtml
fig = oni_periodos_base(pd_normalidad.query("range_end>1964"),
                        pd_temperatura.query("range_end>1964")
                        )

fig.write_image(DIR + 'figures/nino34/climatologia.png')
fig.show()

In [16]:
# Calculando del ONI
pd_oni_test = pd_temperatura.copy()
pd_oni_test['anomalias'] = pd_oni_test['nino34_mean'] - pd_oni_test['climatologica']

pd_oni_test['oni'] =  pd_oni_test.anomalias.rolling(min_periods=1, window=3, center=True).mean()
pd_oni_test['periodo'] =  pd.to_datetime(pd_oni_test['periodo'],format='%Y-%m',errors='ignore')

pd_oni_test = pd_oni_test.sort_values('periodo',ascending=True)
pd_oni_test.index = pd.to_datetime(pd_oni_test.periodo)
pd_oni_test.head()

,year,month,mes,mes_year,nino34_mean,periodo,anomalia_periodo,date_init,date_end,climatologica,range_init,range_end,base,anomalias,oni
periodo,,,,,,,,,,,,,,,
1950-01-01,1950,1,enero,enero-1950,24.56,1950-01-01,0,1946,1950,26.18,1916,1945,1916-1945,-1.62,-1.470000
1950-02-01,1950,2,febrero,febrero-1950,25.07,1950-02-01,0,1946,1950,26.39,1916,1945,1916-1945,-1.32,-1.336667
1950-03-01,1950,3,marzo,marzo-1950,25.88,1950-03-01,0,1946,1950,26.95,1916,1945,1916-1945,-1.07,-1.163333
1950-04-01,1950,4,abril,abril-1950,26.29,1950-04-01,0,1946,1950,27.39,1916,1945,1916-1945,-1.10,-1.180000
1950-05-01,1950,5,mayo,mayo-1950,26.19,1950-05-01,0,1946,1950,27.56,1916,1945,1916-1945,-1.37,-1.070000


In [17]:
fig = grafico_oni(pd_oni_test)
fig.write_image(DIR + 'figures/nino34/oni.png')
fig.show()